In [9]:
import sys
sys.path.append('..')

In [10]:
import gdspy
import numpy as np
from importlib import reload
#import libraries.conformal_mapping as cm
#import libraries.general_design_functions as gdf
#import libraries.general_sample_creator as creator
#from  ..libraries import general_design_functions as gdf
#fr
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator

In [11]:
watch=lambda: gdspy.LayoutViewer(depth=0,pattern={'default': 8},background='#FFFFFF') #this opens a viewer

In [12]:
tl_core = 20
tl_gap = 12
tl_ground = 10

resonator_core = 8
resonator_gap = 7
resonator_ground = 16

pad_offset = 1000
pad_element_offset = 1000

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 300
num_couplers = 1

# Transmission line with resonator

In [13]:
reload(gdspy)
reload(creator)
# reload(elements)

layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16
}
sample = creator.Sample('4_qubits',layers_configuration)

#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=9.7e3

central_line_y = sample.chip_geometry.sample_vertical_size/2

ChichkovAB = elements.AirBridge(width=15, 
                                length=60, 
                                padsize=30, 
                                min_spacing = 30,
                                layer_configuration=sample.layer_configuration)

chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads)
sample.add(chip_edge_ground)

grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
                                  width=4, period=500)
sample.add(grid_ground)

# 1. Create contact pads:

p1 = elements.Pad('p1', (pad_offset,  central_line_y), np.pi, tl_core, tl_gap, tl_ground, 
                  layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                  **elements.default_pad_geometry())
p2 = elements.Pad('p2', (sample.chip_geometry.sample_horizontal_size-pad_offset,  central_line_y), 
                  0, tl_core, tl_gap, tl_ground, 
                  layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                  **elements.default_pad_geometry())
sample.add(p1)
sample.add(p2)

# 2. Create main copler:

main_coupler = elements.CPWCoupler('TL-resonator coupler', [(coupler_start, central_line_y), (coupler_start+coupler_length, central_line_y)],
                                   [resonator_core, resonator_ground, tl_core], [resonator_gap, resonator_gap, tl_gap, tl_gap],
                                   tl_ground, sample.layer_configuration, r=100)
sample.add(main_coupler)

# 3. Create fanout to create closed enfd of resonator
fanout_for_closed_end = sample.fanout(o=main_coupler, port='port1', name='closed end resonator fanout', grouping=[1, 3])

# 4. 
g1 = sample.ground(o=fanout_for_closed_end, port='center', name='cl1', grounding_width=10, grounding_between = [(2,3)])

# 5. Create left part of transmission line
left_TL = sample.connect_cpw(o1=p1, o2=g1, port1='port', port2='narrow', name='left TL', points=[])

# 6. Create closed meander of resonator
closed_end_meander = sample.connect_meander(name='closed end', o1=fanout_for_closed_end, port1='up',
                                            meander_length=3300,restricted_scale=400, constant_scale=900, radius=25)

# 7. Create fanout to create closed enfd of resonator
fanout_for_open_end = sample.fanout(o=main_coupler, port='port2', name='open end resonator fanout', grouping=[1, 3])

# 8. 
g2 = sample.ground(o=fanout_for_open_end, port='center', name='cl2', grounding_width=10, grounding_between = [(0,1)])

# 9. Create left part of transmission line
right_TL = sample.connect_cpw(o1=p2, o2=g2, port1='port', port2='narrow', name='right TL', points=[])

# 10. Create closed meander of resonator
open_end_resonator = elements.CPW(name='open end resonator', points=[fanout_for_open_end.get_terminals()['down'].position, (fanout_for_open_end.get_terminals()['down'].position[0],
                                                                                                                    fanout_for_open_end.get_terminals()['down'].position[1]-1000)],
                                  w=fanout_for_open_end.get_terminals()['down'].w, s=fanout_for_open_end.get_terminals()['down'].s,
                                  g=fanout_for_open_end.get_terminals()['down'].g, layer_configuration=sample.layer_configuration, r=25)
sample.add(open_end_resonator)
#smth wrong
# 11. Create grounding of resonator
resonator_ground = sample.ground(o=closed_end_meander, port='port2', name='resonator ground', grounding_width=30, grounding_between = [(0,2)])

In [14]:
design = sample.draw_design()
watch()

/home/alena/anaconda3/lib/python3.7/site-packages/gdspy/path.py:828: UserWarning: [GDSPY] Possible inconsistency found in `FlexPath` due to sharp corner.
  "[GDSPY] Possible inconsistency found in "
/home/alena/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  """Entry point for launching an IPython kernel.


<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [68]:
gdspy.write_gds('1qubit.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)

In [77]:
resonator_ground

RectGrounding resonator ground

In [15]:
sys, connections, elements_ = sample.get_tls()
f, delta, modes = sys.get_modes()
f/(2*np.pi), delta

(array([4.59291001e+09, 8.53551647e+09, 9.78795780e+09, 1.46445793e+10,
        1.53362550e+10, 2.37391775e+10, 2.87444736e+10, 3.01944184e+10,
        5.69020437e+10, 6.09110624e+10, 6.43473729e+10, 9.36927263e+10,
        9.62754665e+10, 1.00822637e+11, 1.17671137e+11, 1.64298503e+11,
        1.68768131e+11, 2.84988691e+11, 2.93955123e+11, 2.97250120e+11,
        2.98143651e+11, 3.45089539e+11, 3.48851308e+11, 5.10152913e+11,
        5.34402985e+11, 5.92093726e+11, 6.24655462e+11, 1.18116067e+12,
        1.23051601e+12, 1.99910827e+12, 2.05289124e+12, 2.07230970e+12,
        2.07755844e+12, 2.66387634e+12, 2.67727014e+12, 2.94390817e+12,
        2.95425226e+12, 2.95778671e+12, 2.95854342e+12, 3.19625180e+12,
        3.20517729e+12, 3.73078459e+12, 3.76826796e+12, 3.78109136e+12,
        3.78343672e+12, 6.54362478e+12, 6.63338361e+12]),
 array([ 4.25632780e+10, -0.00000000e+00,  8.57913099e+10,  1.09757506e+10,
         8.94731102e+09, -0.00000000e+00, -0.00000000e+00,  2.79816810e+11

In [16]:
connections

{(CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 0): 1,
 (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 0): 1,
 (CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 1): 2,
 (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 1): 2,
 (CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 2): 3,
 (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 2): 3,
 (RectFanout closed end resonator fanout, n=3, grouping=(0, 2),
  'center',
  0): 4,
 (RectGrounding cl1, 'wide', 0): 4,
 (RectFanout closed end resonator fanout, n=3, grouping=(0, 2),
  'center',
  1): 5,
 (RectGrounding cl1, 'wide', 1): 5,
 (CPW "left TL", l=968.634, 'port1', 0): 6,
 (<QCreator.elements.pad.Pad at 0x7fde4cec9a90>, 'port', 0): 6,
 (CPW "left TL", l=968.634, 'port2', 0): 7,
 (RectGrounding cl1, 'narrow', 0): 7,
 (CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port2', 0): 8,
 (RectFanout open end resonator fanout, n=3, grouping=(

In [18]:
elements_

{'chip_edge_ground': [],
 'p1': [Port p1],
 'p2': [Port p2],
 'TL-resonator coupler': [TL TL-resonator coupler (n=3)],
 'closed end resonator fanout': [TL closed end resonator fanout_group1 (n=2),
  TL closed end resonator fanout_group2 (n=1)]}

In [19]:
fanout_for_closed_end.w

[20, 16, 8]

In [20]:
fanout_for_closed_end.get_terminals()

{'wide': Type: mc-cpw, Position: (2000.0, 2350.0), Orientation: 3.142
 w: [8, 16, 20], s: [7, 7, 12, 12], g: 10
 reversed: True,
 'center': Type: mc-cpw, Position: (1978.0, 2357.5), Orientation: 0.0
 w: [20, 16], s: [12, 12, 7], g: 10
 reversed: True,
 'up': Type: cpw, Position: (1989.0, 2299.0), Orientation: 1.571
 w: 8, s: 7, g: 10
 reversed: True}

In [27]:
sample.connections

[((CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 0),
  (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 0)),
 ((CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 1),
  (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 1)),
 ((CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 2),
  (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 2))]

In [ ]:
[((CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 0),
  (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 0)),
 ((CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 1),
  (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 1)),
 ((CPWCoupler "TL-resonator coupler", n=3, l=299.798, 'port1', 2),
  (RectFanout closed end resonator fanout, n=3, grouping=(0, 2), 'wide', 2))]